In [2]:
import numpy as np
from codetiming import Timer
from ortools.sat.python import cp_model
import time

model = cp_model.CpModel()

# Define the given grid
z = -1
input_grid = np.array([
    [z, z, z, z, z, z, z, 2, z],
    [z, z, z, z, 2, z, z, z, 5],
    [z, 2, z, z, z, z, z, z, z],
    [z, z, 0, z, z, z, z, z, z],
    [z, z, z, z, z, z, z, z, z],
    [z, z, z, 2, z, z, z, z, z],
    [z, z, z, z, 0, z, z, z, z],
    [z, z, z, z, z, 2, z, z, z],
    [z, z, z, z, z, z, 5, z, z],
], dtype=np.int32)

known = list(np.unique(input_grid[input_grid >= 0]))
omitable = [d for d in range(10) if d not in known]
gcd_lower_bound = 1
gcd_upper_bound = 98765432
num_upper_bound = 987654320
P = 10 ** np.arange(8, -1, -1, dtype=np.int32)

# Define model variables
X = np.array([[model.NewIntVar(0, 9, f"x[{i},{j}]") for j in range(9)] for i in range(9)])
N = np.array([model.NewIntVar(0, num_upper_bound, f"n[{i}]") for i in range(9)])
o = model.NewIntVar(0, 9, "o")
g = model.NewIntVar(gcd_lower_bound, gcd_upper_bound, "g")

# Ensure the omitted digit does not appear in the grid
for i in range(9):
    for j in range(9):
        model.Add(X[i, j] != o)

# Apply constraints for given values
for i in range(9):
    for j in range(9):
        if input_grid[i, j] >= 0:
            model.Add(X[i, j] == input_grid[i, j])

# Define allowed assignments for omitted digit and GCD
def find_divisors(n):
    for i in range(1, int(n**0.5) + 1):  # Loop up to sqrt(n)
        if n % i == 0:
            yield i
            if i != n // i:
                yield n // i      # Yield the paired divisor

allowed_assignments = []
for o_ in range(10):
    for g_ in find_divisors(111111111 * (45 - o_)):
        allowed_assignments.append([o_, g_])

model.AddAllowedAssignments([o, g], allowed_assignments)

# Apply Sudoku constraints (all different in rows, columns, and 3x3 squares)
for i in range(9):
    model.AddAllDifferent(X[i, :])  # Row constraint
    model.AddAllDifferent(X[:, i])  # Column constraint

# 3x3 square constraints
for i in range(0, 9, 3):
    for j in range(0, 9, 3):
        model.AddAllDifferent([
            X[i + dx, j + dy]
            for dx in range(3)
            for dy in range(3)
        ])

# Divisibility constraints
for i in range(9):
    row_number = sum(X[i, j] * P[j] for j in range(9))
    model.Add(N[i] == row_number)
    model.Add(g <= N[i])
    model.AddModuloEquality(0, N[i], g)

# Objective to maximize the GCD
model.Maximize(g)

# Solve problem
# ----------------------------------------------------------------------
status_names: dict[int, str] = {
    cp_model.UNKNOWN: "UNKNOWN",
    cp_model.MODEL_INVALID: "MODEL_INVALID",
    cp_model.FEASIBLE: "FEASIBLE",
    cp_model.INFEASIBLE: "INFEASIBLE",
    cp_model.OPTIMAL: "OPTIMAL",
}

start = time.time()

print(model.validate())
with Timer(initial_text="Solving model..."):
    solver = cp_model.CpSolver()
    status: int = solver.solve(model)
print(f"Status = {status_names[status]}")

end = time.time()

if status in [cp_model.OPTIMAL, cp_model.FEASIBLE]:
    xm = np.array([[solver.value(X[i, j]) for j in range(9)] for i in range(9)], dtype=np.int32)
    ns = np.array([solver.value(N[i]) for i in range(9)])
    print(f"Maximum of objective function: {solver.objective_value}")
    print("Solution = \n", xm, sep="")
else:
    print("No solution found.")



Solving model...
Elapsed time: 4.4394 seconds
Status = OPTIMAL
Maximum of objective function: 12345679.0
Solution = 
[[3 9 5 0 6 1 7 2 8]
 [0 6 1 7 2 8 3 9 5]
 [7 2 8 3 9 5 0 6 1]
 [9 5 0 6 1 7 2 8 3]
 [2 8 3 9 5 0 6 1 7]
 [6 1 7 2 8 3 9 5 0]
 [8 3 9 5 0 6 1 7 2]
 [5 0 6 1 7 2 8 3 9]
 [1 7 2 8 3 9 5 0 6]]
